In [1]:
#include "../common.hpp"
#include <initializer_list>

using namespace std;

# auto

- Prior to C++11, the keyword `auto` was a storage class specifier
    - But rarely used

In [2]:
void f() {
    auto int i = 42;
}

input_line_9:2:5: warning: 'auto' storage class specifier is not permitted in C++11, and will not be supported in future releases [-Wauto-storage-class]
    auto int i = 42;
    ^~~~~


## auto variable declarations

`auto` can be used in a variable declaration.

- `auto` alone will always deduce to an object type (not a reference, or const type)

In [3]:
const int& a = 42;
auto b = a;
    
type_name<decltype(a)>();
type_name<decltype(b)>();

int const&
int


- cv-qualifiers and reference modifiers can be combined with auto

In [4]:
const auto c = b;
const auto& d = b;

type_name<decltype(c)>();
type_name<decltype(d)>();

int const
int const&


- `decltype(auto)` can be used to declare a variable which matches the expression type

In [5]:
decltype(auto) e = a;

type_name<decltype(e)>();

int const&


- `auto&&` is a forwarding reference
- `decltype` is used with the `std::forward` template argument

In [6]:
namespace {

annotate some_function() { return annotate(); }
void some_sink(annotate) { }
    
} // namespace

In [7]:
auto&& q = some_function();
some_sink(forward<decltype(q)>(q));

annotate ctor
annotate move-ctor
annotate dtor


## auto function results

C++11 added trailing return types. This allows a return type which is dependent on an argument type.

In [8]:
namespace {
    
int function_a();
auto function_b() -> int; // same signature as function_a

template <class T, class U>
auto add(T a, U b) -> decltype(a + b) {
    return a + b;
}
    
} // namespace

- C++14 allows the return type to be omitted for `template` and `inline` functions.

In [9]:
inline auto mixed_add(unsigned a, signed b) {
    return a + b;
}

auto r = mixed_add(42, -1);

In [10]:
type_name<decltype(r)>();
cout << r << "\n";

unsigned int
41


- Using `decltype(auto)` for the return type preserves references and cv-qualifiers

In [11]:
namespace {
    
vector<int> v = { 0, 1, 2, 3 };

inline decltype(auto) back() {
    return v.back();
}
    
} // namespace

In [12]:
type_name<decltype(back())>();

int&


## auto template parameters (C++17)

- C++17 adds auto template parameters.

In [13]:
namespace cpp14 {

template <class T, T N>
struct integral_constant {
      static constexpr T value = N;
};
    
using true_type = integral_constant<bool, true>;

}

- In C++17 this can be implemented without specifying the type.

```cpp
template <auto N>
struct integral_constant {
      static constexpr decltype(N) value = N;
};

using true_type = integral_constant<true>;
```

## structured bindings (C++17)

- _structured bindings_ allow you to construct a set of objects from an array, tuple, _type like_, or class/struct public members

```cpp
auto [first_name, last_name] = make_pair("Jane"s, "Smith"s);
cout << last_name << ", " << first_name << '\n';
```
---
```
Smith, Jane
```

- simple handling of functions returning structures

```cpp
int a[] = { 0, 3, 4, 5, 5, 5, 6, 6, 7 };

for(auto [f, l] = equal_range(begin(a), end(a), 5); f != l; ++f) {
    cout << *f << '\n';
}
```
---
```
5
5
5
```

- deduction rules are different from other uses of `auto`
    - each element is deduced to `decltype(e)` where is the corresponding member
    
```cpp
int a = 1, b = 2;
auto [x, y] = tie(a, b);

x = 42;
cout << a << endl;
```
---
```
42
```

```cpp
type_name<decltype(x)>();
```
---
```
int&
```

- can use const and reference qualifiers with structured bindings

```cpp
int a = 1, b = 2;
const auto& [x, y] = tie(a, b);

a = 42;
cout << x << endl;
```
---
```
42
```

### limitations of structured bindings

- no way to ignore an element
- no way to reorder, or bind by name

## auto lambda arguments (covered in lambda section)

## Issue

### Qualify with references appropriately

In [14]:
vector<annotate> w(5);

annotate ctor
annotate ctor
annotate ctor
annotate ctor
annotate ctor


In [15]:
for (auto e : w) {
    // do something
}

annotate copy-ctor
annotate dtor
annotate copy-ctor
annotate dtor
annotate copy-ctor
annotate dtor
annotate copy-ctor
annotate dtor
annotate copy-ctor
annotate dtor


In [16]:
for (const auto& e : w) {
    // do something
}

## recommendations for auto variables

- Use auto where required
- Use auto where complex names do not add clarity
```cpp
for (typename vector<my_class>::const_iterator f = v.begin(), l = v.end;
     f != l; ++f) {
```
```cpp
for (auto f = v.begin(), l = v.end(); f != l; ++f) {
```
- `auto` can make code less brittle to small changes (like size of integral result)
- Use for structured binding (better than any alternative)


- Otherwise it becomes a matter of taste. Some people recommend using `auto` almost never, some people recommend using `auto` always. If it makes your code more clear, use it, otherwise do not, but avoid bikeshedding.

## recommendations for auto function results

- Use trailing result types except for `bool` and `void`
```cpp
auto some_function() -> some_type;
bool is_something();
void got_nothin();
```
    - code aligns better, and it is easier to read
    - avoids having trailing result types breaking the style when required
- Don't use auto results unless required, or naming the type adds nothing
```cpp
auto some_function() {
    //...
    return f(g(a + y)); // don't make me figure out this type!
}
```
    - Nearly every time I've omitted the result, I've regretted it 

## homework

- Experiment with `auto` variables and trailing return types in your project
- Find some cases where you think it improves the code readability
- Find some cases where you think it hinders the code readability
- Write up examples of both on the [wiki](git.corp.adobe.com/better-code/class/wiki)